# XBeach Output Configuration Tutorial

This notebook demonstrates how to configure output for XBeach models using the 
`Output` class. XBeach supports four types of output:

1. **Time-averaged spatial output** (`meanvars`) - Statistics over time intervals
2. **Instantaneous spatial output** (`globalvars`) - Snapshots at specific times
3. **Fixed point output** (`pointvars` + `points`) - Time series at specific locations
4. **Run-up gauge output** (`pointvars` + `rugauges`) - Time series at moving waterline

Let's start by importing the necessary modules.

In [27]:
from rompy_xbeach.components.output import Output
from rompy_xbeach.types import OutputVarsEnum

def show_params(output):
    """Helper function to display params in XBeach params.txt format."""
    params = output.params
    for key, value in params.items():
        if isinstance(value, list):
            # For lists, print each item on its own line
            for item in value:
                print(item)
        else:
            # For regular values, print key = value
            print(f"{key} = {value}")

## 1. Basic Output Format Configuration

Let's start by configuring the output file format and name. The default output 
format is NetCDF, which is recommended for most applications.

In [28]:
output_format = Output(
    outputformat="netcdf",  # Default, can also be "fortran" or "debug"
    ncfilename="my_simulation.nc",
    outputprecision="double",
)

show_params(output_format)

outputformat = netcdf
outputprecision = double
ncfilename = my_simulation.nc


**Available options:**
- `outputformat`: `"netcdf"` (default), `"fortran"`, or `"debug"`
- `outputprecision`: `"single"` or `"double"` (for NetCDF only)
- `ncfilename`: Name of the NetCDF output file (default: `xboutput.nc`)

## 2. Time-Averaged Spatial Output (Mean Variables)

Mean variables provide time-averaged statistics (mean, variance, min, max) over 
specified intervals. This is useful for understanding average conditions.

In [29]:
output_mean = Output(
    ncfilename="output.nc",
    meanvars=[
        "H",          # Wave height
        "thetamean",  # Mean wave direction
        "u",          # Velocity in x-direction
        "v",          # Velocity in y-direction
        "zs",         # Water level
        "zb",         # Bed level
    ],
    tstart=0.0,      # Start output at t=0
    tintm=3600.0,    # Output interval: 1 hour
)

show_params(output_mean)

outputformat = netcdf
ncfilename = output.nc
tstart = 0.0
tintm = 3600.0
nmeanvar = 6
H
thetamean
u
v
zs
zb


**Key points:**
- `nmeanvar` is automatically calculated from the list length
- `tintm` defines the averaging interval (seconds)
- First output is at `tstart + tintm`
- The output represents the average over `[tstart, tstart+tintm]`

## 3. Instantaneous Spatial Output (Global Variables)

Global variables provide instantaneous snapshots of the entire model domain at 
specified time intervals.

In [30]:
output_global = Output(
    ncfilename="output.nc",
    globalvars=[
        "H",   # Wave height
        "zs",  # Water level
        "u",   # Velocity x
        "v",   # Velocity y
    ],
    tstart=0.0,
    tintg=10.0,  # Output every 10 seconds
)

show_params(output_global)

outputformat = netcdf
ncfilename = output.nc
tstart = 0.0
tintg = 10.0
nglobalvar = 4
H
zs
u
v


**Key points:**
- `nglobalvar` is automatically calculated
- `tintg` defines the output interval (seconds)
- First output is at `tstart`
- Provides instantaneous values (not averaged)

## 4. Fixed Point Output

Point output provides time series at specific fixed locations in the model domain.
This is useful for detailed analysis at locations of interest.

In [31]:
output_points = Output(
    ncfilename="output.nc",
    points=[
        (0.0, 500.0),      # Point 1: x=0m, y=500m
        (1000.0, 500.0),   # Point 2: x=1000m, y=500m
        (2000.0, 500.0),   # Point 3: x=2000m, y=500m
    ],
    pointvars=[
        "H",   # Wave height
        "u",   # Velocity x
        "v",   # Velocity y
        "zs",  # Water level
    ],
    tstart=0.0,
    tintp=5.0,  # Output every 5 seconds
)

show_params(output_points)

outputformat = netcdf
ncfilename = output.nc
tstart = 0.0
tintp = 5.0
npoints = 3
0.0 500.0
1000.0 500.0
2000.0 500.0
npointvar = 4
H
u
v
zs


**Key points:**
- Points are specified as `(x, y)` coordinate pairs
- XBeach links each point to the nearest computational grid point
- `npoints` and `npointvar` are automatically calculated
- `tintp` defines the output interval (defaults to `tintg` if not specified)
- Coordinates are formatted as space-separated strings in params

## 5. Run-up Gauge Output

Run-up gauges track the moving waterline along cross-shore transects. This is 
particularly useful for studying run-up and swash processes.

In [32]:
output_rugauges = Output(
    ncfilename="output.nc",
    rugauges=[
        (0.0, 500.0),    # Gauge 1: transect at y=500m
        (0.0, 1000.0),   # Gauge 2: transect at y=1000m
    ],
    pointvars=[
        "H",   # Wave height
        "zs",  # Water level
    ],
    nrugdepth=5,  # Number of depths to compute runup
    tstart=0.0,
    tintp=1.0,  # Output every second
)

show_params(output_rugauges)

outputformat = netcdf
ncfilename = output.nc
nrugdepth = 5
tstart = 0.0
tintp = 1.0
nrugauge = 2
0.0 500.0
0.0 1000.0
npointvar = 2
H
zs


**Key points:**
- Run-up gauges share point output variables (`pointvars`)
- XBeach automatically adds `xw`, `yw`, and `zs` to the output
- Gauges are linked to the nearest cross-shore transect (not just a point)
- `nrugdepth` specifies how many water depths to track
- Useful for studying swash zone dynamics

## 6. Output Timing Control

XBeach provides two ways to control when output is written: fixed intervals or 
file-based times.

### 6.1 Fixed Interval Timing

The most common approach is to use fixed time intervals for output. Each output 
type has its own timing parameter:

In [33]:
output_timing = Output(
    ncfilename="output.nc",
    globalvars=["H", "zs"],
    meanvars=["H", "u"],
    pointvars=["H"],
    points=[(0.0, 500.0)],
    tstart=0.0,      # Start time for output (seconds)
    tintg=10.0,      # Global output interval (seconds)
    tintm=3600.0,    # Mean output interval (seconds)
    tintp=5.0,       # Point output interval (seconds)
)

show_params(output_timing)

outputformat = netcdf
ncfilename = output.nc
tstart = 0.0
tintg = 10.0
tintm = 3600.0
tintp = 5.0
npoints = 1
0.0 500.0
nmeanvar = 2
H
u
nglobalvar = 2
H
zs
npointvar = 1
H


**Timing parameters:**
- `tstart`: Start time for output (default: 0.0)
- `tintg`: Interval for global (instantaneous) output (default: 1.0 s)
  - First output at `tstart`
- `tintm`: Interval for mean output (default: `tstop - tstart`)
  - First output at `tstart + tintm`
  - Represents average over `[tstart, tstart + tintm]`
- `tintp`: Interval for point/runup output (default: same as `tintg`)
  - First output at `tstart`

### 6.2 File-Based Timing

For irregular output schedules or specific snapshots, you can specify output times 
from files. This is useful for event-based output or when you need precise control.

In [ ]:
output_file_times = Output(
    ncfilename="output.nc",
    globalvars=["H", "zs"],
    tsglobal="global_times.txt",  # File with global output times
    meanvars=["H", "u"],
    tsmean="mean_times.txt",      # File with mean output times
)

show_params(output_file_times)

**File-based timing parameters:**
- `tsglobal`: File containing global output times
- `tsmean`: File containing mean output times
- `tspoint`: File containing point output times

**Important notes:**
- Each file should contain the number of times followed by one time value per line (in seconds)
- File-based times **supersede** fixed interval times if both are specified
- Times are relative to the simulation start
- Useful for irregular sampling or event-based output

## 7. Progress Output Control

Control whether XBeach displays progress information during the simulation.

In [34]:
output_progress = Output(
    ncfilename="output.nc",
    timings=True,  # Enable progress output to screen
    globalvars=["H"],
    tintg=10.0,
)

show_params(output_progress)

outputformat = netcdf
ncfilename = output.nc
timings = 1
tintg = 10.0
nglobalvar = 1
H


**Note:** The `timings` boolean is automatically converted to an integer (0 or 1) 
for XBeach compatibility.

## 8. Comprehensive Configuration

Let's put it all together with a comprehensive output configuration that uses 
multiple output types simultaneously.

In [38]:
output_comprehensive = Output(
    # Output file configuration
    outputformat="netcdf",
    ncfilename="comprehensive_output.nc",
    outputprecision="double",
    
    # Time-averaged spatial output
    meanvars=[
        "H",
        "u",
        "v",
    ],
    
    # Instantaneous spatial output
    globalvars=[
        "zs",
    ],
    
    # Fixed point output
    points=[
        (0.0, 500.0),
        (1000.0, 500.0),
        (2000.0, 500.0),
    ],
    
    # Run-up gauges
    rugauges=[
        (0.0, 750.0),
        (0.0, 1250.0),
    ],
    
    # Point/runup variables (shared)
    pointvars=[
        "H",
        "u",
        "v",
        "zs",
    ],
    
    # Timing configuration
    tstart=0.0,
    tintg=10.0,    # Global output every 10 seconds
    tintm=3600.0,  # Mean output every hour
    tintp=5.0,     # Point output every 5 seconds
    
    # Run-up configuration
    nrugdepth=5,
    
    # Progress output
    timings=True,
)

show_params(output_comprehensive)

outputformat = netcdf
outputprecision = double
ncfilename = comprehensive_output.nc
nrugdepth = 5
timings = 1
tstart = 0.0
tintg = 10.0
tintm = 3600.0
tintp = 5.0
npoints = 3
0.0 500.0
1000.0 500.0
2000.0 500.0
nrugauge = 2
0.0 750.0
0.0 1250.0
nmeanvar = 3
H
u
v
nglobalvar = 1
zs
npointvar = 4
H
u
v
zs


## 9. Validation and Warnings

The Output class includes validators that warn about potential issues. Let's see 
some examples.

### Example 1: Exceeding Variable Limits

XBeach has limits on the number of variables for each output type. The Output class 
will warn you if you exceed these limits.

In [54]:
# Build a list with all supported variables
all_variables = [v.value for v in OutputVarsEnum]

# This will generate a warning (limit is 15 mean variables)
output_too_many = Output(
    meanvars=all_variables[:20]  # 20 variables exceeds the limit
)

show_params(output_too_many)

2025-10-08 22:57:36 [WARNING] rompy_xbeach.components.output: ! More than 15 meanvars requested. XBeach only supports up to 15. Beware of possible unexpected results in the model.


outputformat = netcdf
nmeanvar = 20
As
BR
Cdrag
D
D15
D50
D50top
D90
D90top
DR
Dc
Df
Dp
Dveg
E
Fvegu
Fvegv
Fx
Fy
H


**Limits:**
- Mean variables: 15
- Global variables: 20
- Point variables: 50
- Points/run-up gauges: 50 each

### Example 2: Inconsistent Point Configuration

If you define point variables without locations (or vice versa), you'll get a warning.

In [48]:
# Warning: pointvars without locations
output_no_locations = Output(
    pointvars=["H", "u"]
    # No points or rugauges defined!
)

show_params(output_no_locations)

2025-10-08 22:51:33 [WARNING] rompy_xbeach.components.output: ! Point output variables (pointvars) are defined, but no point locations (points) or runup gauge locations (rugauges) have been prescribed. Output will not be generated.


outputformat = netcdf
npointvar = 2
H
u


In [49]:
# Warning: locations without variables
output_no_vars = Output(
    points=[(0.0, 500.0)]
    # No pointvars defined!
)

show_params(output_no_vars)

2025-10-08 22:51:48 [WARNING] rompy_xbeach.components.output: ! Point locations (points) or runup gauge locations (rugauges) are defined, but no point output variables (pointvars) have been prescribed. No point/runup output will be generated.


outputformat = netcdf
npoints = 1
0.0 500.0


### Example 3: Conflicting Time Specifications

If you specify both fixed intervals and file-based times, the file-based times 
will supersede the fixed intervals, and you'll get a warning.

In [50]:
# Warning: both fixed and file times specified
output_conflict = Output(
    globalvars=["H"],
    tintg=10.0,              # Fixed interval
    tsglobal="times.txt",    # File-based times (this takes precedence)
)

show_params(output_conflict)

2025-10-08 22:52:05 [WARNING] rompy_xbeach.components.output: ! Global times defined by both fixed (tintg) and file (tsglobal) times. The file-based times (tsglobal) will supersede the fixed interval.


outputformat = netcdf
tintg = 10.0
tsglobal = times.txt
nglobalvar = 1
H


## 10. Available Output Variables

Here's a complete list of all output variables supported in rompy-xbeach:

In [51]:
print("XBeach Output Variables (OutputVarsEnum):")
print("=" * 60)
for var in OutputVarsEnum:
    print(f"  {var.value}")

XBeach Output Variables (OutputVarsEnum):
  As
  BR
  Cdrag
  D
  D15
  D50
  D50top
  D90
  D90top
  DR
  Dc
  Df
  Dp
  Dveg
  E
  Fvegu
  Fvegv
  Fx
  Fy
  H
  Hrunup
  L1
  Qb
  R
  Sk
  Subg
  Susg
  Sutot
  Svbg
  Svsg
  Svtot
  Sxx
  Sxy
  Syy
  Tbore
  Tsg
  alfau
  alfav
  alfaz
  bedfriccoef
  bi
  breaking
  bwalpha
  c
  ca
  ccg
  cctot
  ccz
  ceqbg
  ceqsg
  cf
  cfu
  cfv
  cg
  cgx
  cgx_s
  cgy
  cgy_s
  cobs
  costh
  costh_s
  ctheta
  ctheta_s
  cx
  cy
  dU
  dUi
  dV
  dassim
  dcbdx
  dcbdy
  dcmdo
  dcsdx
  dcsdy
  depo_ex
  depo_im
  dinfil
  dnc
  dnu
  dnv
  dnz
  dobs
  dsc
  dsdnui
  dsdnvi
  dsdnzi
  dsu
  dsv
  dsz
  dzav
  dzbdt
  dzbdx
  dzbdy
  dzbed
  dzbnow
  dzs0dn
  dzsdt
  dzsdx
  dzsdy
  ee
  ee_s
  ero
  fw
  gw0back
  gwbottom
  gwcurv
  gwhead
  gwheadb
  gwheight
  gwlevel
  gwqx
  gwqy
  gwu
  gwv
  gww
  hh
  hhw
  hhwcins
  hhws
  hold
  hu
  hum
  hv
  hvm
  idrift
  infil
  istruct
  iwl
  jdrift
  k
  kb
  kturb
  maxzs
  minzs
  n
  n

## 11. Integration with XBeach Config

The Output object is typically used as part of a larger XBeach configuration:

In [52]:
# Example of how Output fits into a Config object
example_usage = """
from rompy_xbeach.config import Config
from rompy_xbeach.components.output import Output

config = Config(
    # ... other configuration parameters ...
    
    output=Output(
        ncfilename="simulation_output.nc",
        meanvars=["H", "zs"],
        globalvars=["H"],
        points=[(0.0, 500.0)],
        pointvars=["H", "u"],
        tstart=0.0,
        tintg=10.0,
        tintm=3600.0,
    ),
    
    # ... other configuration parameters ...
)

# The output.params will be merged into the params.txt file
params = config.output.params
"""

print("Integration Example:")
print(example_usage)

Integration Example:

from rompy_xbeach.config import Config
from rompy_xbeach.components.output import Output

config = Config(
    # ... other configuration parameters ...

    output=Output(
        ncfilename="simulation_output.nc",
        meanvars=["H", "zs"],
        globalvars=["H"],
        points=[(0.0, 500.0)],
        pointvars=["H", "u"],
        tstart=0.0,
        tintg=10.0,
        tintm=3600.0,
    ),

    # ... other configuration parameters ...
)

# The output.params will be merged into the params.txt file
params = config.output.params



## Summary

Key takeaways:

1. **Four output types**: mean, global, point, and run-up gauge
2. **Automatic counting**: `nmeanvar`, `nglobalvar`, etc. are calculated automatically
3. **Flexible timing**: Use fixed intervals or file-based times
4. **Validation**: Built-in warnings for common configuration issues
5. **Simple to complex**: Start simple and add complexity as needed
6. **Type-safe**: Uses enums for variable names to prevent typos

For more information, see the XBeach manual and the Output class documentation.

## Next Steps

- Explore other XBeach configuration components (grid, boundary conditions, etc.)
- Run a complete simulation with your output configuration
- Analyse the output NetCDF file using xarray or other tools
- Experiment with different output variables and timing configurations